<a href="https://colab.research.google.com/github/omarmhaimdat/SoundClassifier/blob/master/SoundClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sound Classifier
![final result](https://raw.githubusercontent.com/omarmhaimdat/SoundClassifier/master/final-result.png)

##Omar MHAIMDAT

* [Linkedin](https://www.linkedin.com/in/omarmhaimdat/)
* **Email:** omarmhaimdat@gmail.com
* [Look at the rest of my repos](https://github.com/omarmhaimdat/)

Distributed under the MIT license.



---



##Install Turicreate

In [0]:
!pip install turicreate

##Making sure to use the GPU

In [0]:
!pip uninstall -y mxnet

In [0]:
!pip install mxnet-cu100==1.4.0.post0

## Unzip the ESC-50 dataset

Make sure to import it.

**Link to download the ZIP file:**  [ESC-50 Github](https://github.com/karoldvl/ESC-50)

In [0]:
!unzip -uq "/content/ESC-50.zip" -d "/content/ESC-50"

#Train the model

In [0]:
import turicreate as tc
from os.path import basename

tc.config.set_num_gpus(-1)

# Load the audio data and meta data.
data = tc.load_audio('/content/ESC-50/ESC-50-master/audio/')
meta_data = tc.SFrame.read_csv('/content/ESC-50/ESC-50-master/meta/esc50.csv')

# Join the audio data and the meta data.
data['filename'] = data['path'].apply(lambda p: basename(p))
data = data.join(meta_data)

# Drop all records which are not part of the ESC-10.
data = data.filter_by('True', 'esc10')

# Make a train-test split, just use the first fold as our test set.
test_set = data.filter_by(1, 'fold')
train_set = data.filter_by(1, 'fold', exclude=True)

# Create the model.
model = tc.sound_classifier.create(train_set,
                                   target='category',
                                   feature='audio',
                                   max_iterations=100,
                                   custom_layer_sizes=[200, 200])

# Generate an SArray of predictions from the test set.
predictions = model.predict(test_set)

# Evaluate the model and print the results
metrics = model.evaluate(test_set)
print(metrics)

# Save the model for later use in Turi Create
model.save('SoundClassification.model')

# Export for use in Core ML
model.export_coreml('SoundClassification.mlmodel')

Finished parsing file /content/ESC-50/ESC-50-master/meta/esc50.csv

Parsing completed. Parsed 100 lines in 0.2471 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,int,int,str,str,int,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /content/ESC-50/ESC-50-master/meta/esc50.csv

Parsing completed. Parsed 2000 lines in 0.017232 secs.

Creating a validation set from 5 percent of training data. This may take a while.
	You can set ``validation_set=None`` to disable validation tracking.

Preprocessing audio data -
Preprocessed 73 of 304 examples
Preprocessed 152 of 304 examples
Preprocessed 232 of 304 examples
Preprocessed 304 of 304 examples

Preparing validataion set

Training a custom neural network -
+-------------------------+-------------------------+-------------------------+-------------------------+
| Iteration               | Training Accuracy       | Validation Accuracy (%) | Elapsed Time            |
+-------------------------+-------------------------+-------------------------+-------------------------+
| 1                       | 0.514                   | 0.391                   | 100.406                 |
+-------------------------+-------------------------+-------------------------+-------------------------+
| 2                       | 0.663                   | 0.531                   | 100.796          